# Detectron2
First we need to install required dependencies.

In [1]:
!apt install git gcc g++ -y
!pip install "git+https://github.com/facebookresearch/detectron2.git"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
git is already the newest version (1:2.17.1-1ubuntu0.9).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-id0edx1s
  Running command git clone --filter=blob:none -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-id0edx1s
  Resolved https://github.com/facebookresearch/detectron2.git to commit e3053c14cfcce515de03d3093c0a7b4e734fa41f
  Preparing metadata (setup.py) ... done


In [2]:
!apt-get install ffmpeg libsm6 libxext6 unzip  -y
!pip install opencv-python scipy scikit-learn scikit-image pandas numpy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
unzip is already the newest version (6.0-21ubuntu1.1).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [3]:
!python3 -m pip install --upgrade pip
!pip uninstall --yes kaggle
!pip install kaggle

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
  Using cached kaggle-1.5.12-py3-none-any.whl


## Kaggle

We need to set the kaggle API key in order to download the dataset.

In [7]:
!mkdir ~/.kaggle

In [8]:
%%writefile ~/.kaggle/kaggle.json
{"username":"maximilianschulze","key":"secretkey"}

Writing /root/.kaggle/kaggle.json


In [9]:
!kaggle competitions download -c sartorius-cell-instance-segmentation 

100%|█████████████████████████████████████▉| 2.31G/2.31G [01:35<00:00, 25.0MB/s]
100%|██████████████████████████████████████| 2.31G/2.31G [01:35<00:00, 26.0MB/s]


In [10]:
!kaggle datasets download -d markunys/livecell-dataset -p livecell

100%|█████████████████████████████████████▉| 4.51G/4.51G [03:02<00:00, 28.0MB/s]
100%|██████████████████████████████████████| 4.51G/4.51G [03:02<00:00, 26.6MB/s]


In [40]:
!unzip sartorius-cell-instance-segmentation.zip > /dev/null

In [ ]:
%cd livecell
!unzip livecell-dataset.zip
%cd ..

## Train detectron

Now that we installed all dependencies and unpacked all the required data we can start to train detectron.

In [4]:
import detectron2
import torch
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
import copy
setup_logger()

<Logger detectron2 (DEBUG)>

In [5]:
!pip install pycocotools
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import pandas as pd
import pycocotools
from tqdm import tqdm
import pickle
import itertools

# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

# From https://newbedev.com/encode-numpy-array-using-uncompressed-rle-for-coco-dataset
def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(itertools.groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

### Load annotations and convert them to COCO format

First we load all the masks and convert them to the coco format.

In [ ]:
if (os.path.exists("out.pickle")):
  with open("out.pickle", "rb") as handle:
    out = pickle.load(handle)
else:
  train_df = pd.read_csv('./train.csv')
  out = []
  # Create unique IDs from the masks
  cat_ids = {name:id+1 for id, name in enumerate(train_df.cell_type.unique())} 
  # For each image which is identifiable by the id create a dict with information about
  # the image and about the segmentation masks
  for index, group in tqdm(train_df.groupby("id")):
      image = {}
      image["file_name"] = f"{index}.png"
      image["height"] = 520
      image["width"] = 704
      image["image_id"] = index
      image["annotations"] = []
      # For each instance we create a mask annotation
      for _, row in group.iterrows():
          annotation = {}
          mk = rle_decode(row["annotation"], (row["height"], row["width"]))
          # Get the x and y coordinates where the mask is present
          ys, xs = np.where(mk)
          # Get the x position of the upper left corner
          x1 = min(xs)
          # Get the x position of the lower right corner
          x2 = max(xs)
          # Get the y position of the upper left corner
          y1 = min(ys)
          # Get the y position of the lower right corner
          y2 = max(ys)
          # Use the coordinates to get the bounding box
          annotation["bbox"] = [int(x1), int(y1), int(x2), int(y2)]

          annotation["bbox_mode"] = 0
          annotation["image_id"] = index
          # Set the category
          annotation["category_id"] = cat_ids[row.cell_type]
          # Encode the segmentation mask using pycocotools
          annotation["segmentation"] = pycocotools.mask.encode(np.asarray(mk, order="F"))
          # annotation["segmentation"]["counts"] = annotation["segmentation"]["counts"].decode("utf8")
          # Append the annotation to the segmentation masks
          image["annotations"].append(annotation)
      out.append(image)
  # save the result in a pickle file so that we don't have to precompute every time
  with open("out.pickle", "wb") as handle:
    pickle.dump(out, handle)

In [6]:
# Filters the annotations so that we get a dataset with only
# one celltype
def getClassOut(classNum: int):
  outClone = [copy.deepcopy(img) for img in out]
  for img in outClone:
    # Set the file_name to the correct path
    img["file_name"] = 'train/' + img["file_name"]
    newAnnotations = []
    for seg in img["annotations"]:
      # Keep only cells with the type we want to filter
      if seg["category_id"] == classNum + 1:
        seg["category_id"] = 1
        newAnnotations.append(seg)
    img["annotations"] = newAnnotations
  return outClone

In [7]:
# Sets the filename of the dataset correctly
def getAllOut():
  outClone = [copy.deepcopy(img) for img in out]
  for img in outClone:
    img["file_name"] = 'train/' + img["file_name"]
  return outClone

### Register datasets

After computation of the annotations we need to register the annotations. We use *10%* of the data as validation data.

In [9]:
#dataDir=Path('../input/sartorius-cell-instance-segmentation/')
#register_coco_instances('sartorius_train',{}, '../input/sartorius-cell-instance-segmentation-coco/annotations_train.json', dataDir)
#register_coco_instances('sartorius_val',{},'../input/sartorius-cell-instance-segmentation-coco/annotations_val.json', dataDir)
DatasetCatalog.clear()
MetadataCatalog.clear()
livecellData = 'livecell/LIVECell_dataset_2021/images'
register_coco_instances('livecell_test', {}, 'livecell/livecell_annotations_test.json', livecellData)
register_coco_instances('livecell_train', {}, 'livecell/livecell_annotations_train.json', livecellData)
register_coco_instances('livecell_val', {}, 'livecell/livecell_annotations_val.json', livecellData)
allOut = getAllOut()
# We use 10% of the data as validation data
DatasetCatalog.register('sartorius_train', lambda: allOut[:int(len(allOut)*0.9)])
DatasetCatalog.register('sartorius_val', lambda: allOut[int(len(allOut)*0.9):])
classOut = {}
for classNum in range(3):
  classOut[classNum] = getClassOut(classNum)
  DatasetCatalog.register(f'sartorius_train_{classNum}', lambda: classOut[classNum][:int(len(classOut[classNum])*0.9)])
  DatasetCatalog.register(f'sartorius_val_{classNum}', lambda: classOut[classNum][int(len(classOut[classNum])*0.9):])


In [10]:
# Evaluation masks taken and modified from
# https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-training
from detectron2.structures import polygons_to_bitmask
def polygon_to_rleLiveCell(polygon, shape=(520, 704)):
    #print(polygon)
    mask = polygons_to_bitmask([np.asarray(polygon) + 0.25], shape[0], shape[1])

    rle = mask_util.encode(np.asfortranarray(mask))
    return rle

# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_atLiveCell(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def scoreLiveCell(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    enc_targs = [polygon_to_rleLiveCell(enc_targ[0]) for enc_targ in enc_targs]
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_atLiveCell(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluatorLiveCELL(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(scoreLiveCell(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class TrainerLiveCell(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluatorLiveCELL(dataset_name)

In [11]:
# Evaluation masks taken and modified from
# https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-training
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_at(threshold, iou):
    matches = iou > threshold
    if len(matches) == 0:
      return 0, 0, 0
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn) if (tp + fp + fn) != 0 else 0
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
          if len(out['instances']) == 0 and len(self.annotations_cache[inp['image_id']]) != 0:
            self.scores.append(0)
          elif len(out['instances']) == 0 and len(self.annotations_cache[inp['image_id']]) == 0:
            self.scores.append(1)
          else:
            targ = self.annotations_cache[inp['image_id']]
            self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

## Pre-Train Detectron2 with LiveCell Data

First we pretrain the Detectron2 model with the LiveCell data. We tried two different configurations, one with a ResNet101 backbone and one with a ResNet 50 backbone. The ResNet50 backbone seems to archieve much better performances.

In [12]:
def trainModelLiveCell():
  epochLength = (len(DatasetCatalog.get('livecell_train')) + len(DatasetCatalog.get('livecell_test'))) // 2
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
  cfg.SOLVER.IMS_PER_BATCH = 2
  
  cfg.DATASETS.TRAIN = ("livecell_train", "livecell_test")
  cfg.DATASETS.TEST = ("livecell_val",)
  cfg.TEST.EVAL_PERIOD = epochLength
  cfg.SOLVER.CHECKPOINT_PERIOD = epochLength
  cfg.DATALOADER.NUM_WORKERS = 2
  # Use the official pretrained weights
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
  cfg.SOLVER.BASE_LR = 0.005
  cfg.SOLVER.MAX_ITER = epochLength * 20
  cfg.SOLVER.STEPS = []
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
  # Set the number of classes to eight as LiveCell includes eight different cell tpes
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
  cfg.OUTPUT_DIR = os.path.join(cfg.OUTPUT_DIR, f'/livecell')
  print(cfg.OUTPUT_DIR)
  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = TrainerLiveCell(cfg) 
  trainer.resume_or_load(resume=False)
  trainer.train()

In [13]:
def trainModelLiveCell50():
  epochLength = (len(DatasetCatalog.get('livecell_train')) + len(DatasetCatalog.get('livecell_test')))
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
  cfg.SOLVER.IMS_PER_BATCH = 2
  
  cfg.DATASETS.TRAIN = ("livecell_train", "livecell_test")
  cfg.DATASETS.TEST = ("livecell_val",)
  cfg.TEST.EVAL_PERIOD = epochLength
  cfg.SOLVER.CHECKPOINT_PERIOD = epochLength // 2
  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
  cfg.SOLVER.BASE_LR = 0.01 * 2 / 16
  cfg.SOLVER.MAX_ITER = epochLength * 20
  cfg.SOLVER.STEPS = []
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
  cfg.OUTPUT_DIR = os.path.join(cfg.OUTPUT_DIR, f'./livecell50')
  print(cfg.OUTPUT_DIR)
  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = TrainerLiveCell(cfg) 
  trainer.resume_or_load(resume=True)
  trainer.train()

In [64]:
trainModelLiveCell50()

[12/29 21:30:38 d2.data.datasets.coco]: Loading livecell/livecell_annotations_train.json takes 15.25 seconds.
WARNING [12/29 21:30:38 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/29 21:30:38 d2.data.datasets.coco]: Loaded 3253 images in COCO format from livecell/livecell_annotations_train.json
[12/29 21:30:51 d2.data.datasets.coco]: Loading livecell/livecell_annotations_test.json takes 7.03 seconds.
WARNING [12/29 21:30:51 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/29 21:30:51 d2.data.datasets.coco]: Loaded 1564 images in COCO format from livecell/livecell_annotations_test.json
./output/./livecell50
[12/29 21:30:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (f

[12/29 21:31:09 d2.data.datasets.coco]: Loading livecell/livecell_annotations_train.json takes 14.55 seconds.
WARNING [12/29 21:31:09 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/29 21:31:09 d2.data.datasets.coco]: Loaded 3253 images in COCO format from livecell/livecell_annotations_train.json
[12/29 21:31:23 d2.data.datasets.coco]: Loading livecell/livecell_annotations_test.json takes 7.06 seconds.
WARNING [12/29 21:31:23 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/29 21:31:23 d2.data.datasets.coco]: Loaded 1564 images in COCO format from livecell/livecell_annotations_test.json
[12/29 21:31:24 d2.data.build]: Removed 0 images with no usable annotations. 4817 images left.
[12/29 21:31:25 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333

[12/29 21:35:24 d2.utils.events]:  eta: 5:21:07  iter: 8259  total_loss: 1.291  loss_cls: 0.3347  loss_box_reg: 0.4186  loss_mask: 0.2677  loss_rpn_cls: 0.08459  loss_rpn_loc: 0.1839  time: 0.2240  data_time: 0.0071  lr: 0.00125  max_mem: 16883M
[12/29 21:35:28 d2.utils.events]:  eta: 5:21:08  iter: 8279  total_loss: 1.192  loss_cls: 0.2704  loss_box_reg: 0.3678  loss_mask: 0.2647  loss_rpn_cls: 0.09439  loss_rpn_loc: 0.1755  time: 0.2242  data_time: 0.0174  lr: 0.00125  max_mem: 16883M
[12/29 21:35:33 d2.utils.events]:  eta: 5:20:29  iter: 8299  total_loss: 1.347  loss_cls: 0.3643  loss_box_reg: 0.431  loss_mask: 0.2874  loss_rpn_cls: 0.09616  loss_rpn_loc: 0.1885  time: 0.2240  data_time: 0.0072  lr: 0.00125  max_mem: 16883M
[12/29 21:35:37 d2.utils.events]:  eta: 5:20:52  iter: 8319  total_loss: 1.257  loss_cls: 0.365  loss_box_reg: 0.3873  loss_mask: 0.264  loss_rpn_cls: 0.08971  loss_rpn_loc: 0.1575  time: 0.2238  data_time: 0.0064  lr: 0.00125  max_mem: 16883M
[12/29 21:35:42 d2.

[12/29 21:37:51 d2.utils.events]:  eta: 5:17:33  iter: 8919  total_loss: 1.228  loss_cls: 0.3056  loss_box_reg: 0.3836  loss_mask: 0.2737  loss_rpn_cls: 0.0683  loss_rpn_loc: 0.1834  time: 0.2237  data_time: 0.0070  lr: 0.00125  max_mem: 16883M
[12/29 21:37:56 d2.utils.events]:  eta: 5:17:29  iter: 8939  total_loss: 1.298  loss_cls: 0.3203  loss_box_reg: 0.4279  loss_mask: 0.2665  loss_rpn_cls: 0.09791  loss_rpn_loc: 0.1814  time: 0.2239  data_time: 0.0216  lr: 0.00125  max_mem: 16883M
[12/29 21:38:01 d2.utils.events]:  eta: 5:17:25  iter: 8959  total_loss: 1.301  loss_cls: 0.3244  loss_box_reg: 0.4138  loss_mask: 0.2855  loss_rpn_cls: 0.083  loss_rpn_loc: 0.1697  time: 0.2240  data_time: 0.0073  lr: 0.00125  max_mem: 16883M
[12/29 21:38:05 d2.utils.events]:  eta: 5:17:33  iter: 8979  total_loss: 1.4  loss_cls: 0.3698  loss_box_reg: 0.4666  loss_mask: 0.3043  loss_rpn_cls: 0.06318  loss_rpn_loc: 0.1851  time: 0.2239  data_time: 0.0068  lr: 0.00125  max_mem: 16883M
[12/29 21:38:10 d2.ut

[12/29 21:40:20 d2.utils.events]:  eta: 5:15:56  iter: 9579  total_loss: 1.252  loss_cls: 0.3449  loss_box_reg: 0.4114  loss_mask: 0.2566  loss_rpn_cls: 0.08127  loss_rpn_loc: 0.1787  time: 0.2238  data_time: 0.0079  lr: 0.00125  max_mem: 16883M
[12/29 21:40:25 d2.utils.events]:  eta: 5:15:55  iter: 9599  total_loss: 1.35  loss_cls: 0.3434  loss_box_reg: 0.4135  loss_mask: 0.274  loss_rpn_cls: 0.1069  loss_rpn_loc: 0.2055  time: 0.2239  data_time: 0.0134  lr: 0.00125  max_mem: 16883M
[12/29 21:40:29 d2.utils.events]:  eta: 5:15:32  iter: 9619  total_loss: 1.269  loss_cls: 0.3184  loss_box_reg: 0.4246  loss_mask: 0.2695  loss_rpn_cls: 0.06672  loss_rpn_loc: 0.1747  time: 0.2239  data_time: 0.0063  lr: 0.00125  max_mem: 16883M
[12/29 21:40:35 d2.data.datasets.coco]: Loading livecell/livecell_annotations_val.json takes 2.52 seconds.
WARNING [12/29 21:40:35 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/29 21:40:35 d2.d

...

[12/30 03:37:51 d2.utils.events]:  eta: 0:09:35  iter: 93699  total_loss: 1.062  loss_cls: 0.2744  loss_box_reg: 0.3698  loss_mask: 0.2389  loss_rpn_cls: 0.05288  loss_rpn_loc: 0.1466  time: 0.2237  data_time: 0.0066  lr: 0.00125  max_mem: 16883M
[12/30 03:37:55 d2.utils.events]:  eta: 0:09:30  iter: 93719  total_loss: 1.214  loss_cls: 0.2794  loss_box_reg: 0.3876  loss_mask: 0.2779  loss_rpn_cls: 0.09147  loss_rpn_loc: 0.18  time: 0.2237  data_time: 0.0071  lr: 0.00125  max_mem: 16883M
[12/30 03:37:59 d2.utils.events]:  eta: 0:09:26  iter: 93739  total_loss: 1.104  loss_cls: 0.2453  loss_box_reg: 0.3345  loss_mask: 0.2689  loss_rpn_cls: 0.06828  loss_rpn_loc: 0.1693  time: 0.2237  data_time: 0.0075  lr: 0.00125  max_mem: 16883M
[12/30 03:38:04 d2.utils.events]:  eta: 0:09:22  iter: 93759  total_loss: 1.054  loss_cls: 0.2407  loss_box_reg: 0.3475  loss_mask: 0.2422  loss_rpn_cls: 0.06383  loss_rpn_loc: 0.1426  time: 0.2237  data_time: 0.0075  lr: 0.00125  max_mem: 16883M
[12/30 03:38:0

[12/30 03:40:19 d2.utils.events]:  eta: 0:07:12  iter: 94359  total_loss: 1.088  loss_cls: 0.2509  loss_box_reg: 0.3494  loss_mask: 0.2643  loss_rpn_cls: 0.05909  loss_rpn_loc: 0.1624  time: 0.2237  data_time: 0.0077  lr: 0.00125  max_mem: 16883M
[12/30 03:40:23 d2.utils.events]:  eta: 0:07:08  iter: 94379  total_loss: 1.213  loss_cls: 0.2768  loss_box_reg: 0.3752  loss_mask: 0.2564  loss_rpn_cls: 0.07053  loss_rpn_loc: 0.1538  time: 0.2237  data_time: 0.0102  lr: 0.00125  max_mem: 16883M
[12/30 03:40:27 d2.utils.events]:  eta: 0:07:04  iter: 94399  total_loss: 1.055  loss_cls: 0.2441  loss_box_reg: 0.3147  loss_mask: 0.2475  loss_rpn_cls: 0.05754  loss_rpn_loc: 0.1583  time: 0.2237  data_time: 0.0081  lr: 0.00125  max_mem: 16883M
[12/30 03:40:32 d2.utils.events]:  eta: 0:07:00  iter: 94419  total_loss: 1.163  loss_cls: 0.2853  loss_box_reg: 0.3648  loss_mask: 0.2586  loss_rpn_cls: 0.06214  loss_rpn_loc: 0.1681  time: 0.2237  data_time: 0.0071  lr: 0.00125  max_mem: 16883M
[12/30 03:40

[12/30 03:42:46 d2.utils.events]:  eta: 0:04:49  iter: 95019  total_loss: 1.156  loss_cls: 0.2992  loss_box_reg: 0.359  loss_mask: 0.2588  loss_rpn_cls: 0.07122  loss_rpn_loc: 0.152  time: 0.2237  data_time: 0.0067  lr: 0.00125  max_mem: 16883M
[12/30 03:42:51 d2.utils.events]:  eta: 0:04:44  iter: 95039  total_loss: 1.113  loss_cls: 0.2765  loss_box_reg: 0.3421  loss_mask: 0.2494  loss_rpn_cls: 0.07668  loss_rpn_loc: 0.1377  time: 0.2237  data_time: 0.0202  lr: 0.00125  max_mem: 16883M
[12/30 03:42:55 d2.utils.events]:  eta: 0:04:40  iter: 95059  total_loss: 1.095  loss_cls: 0.2528  loss_box_reg: 0.3645  loss_mask: 0.2491  loss_rpn_cls: 0.06525  loss_rpn_loc: 0.135  time: 0.2237  data_time: 0.0076  lr: 0.00125  max_mem: 16883M
[12/30 03:43:00 d2.utils.events]:  eta: 0:04:35  iter: 95079  total_loss: 1.295  loss_cls: 0.3293  loss_box_reg: 0.4075  loss_mask: 0.2789  loss_rpn_cls: 0.08958  loss_rpn_loc: 0.1871  time: 0.2237  data_time: 0.0075  lr: 0.00125  max_mem: 16883M
[12/30 03:43:04

[12/30 03:45:13 d2.utils.events]:  eta: 0:02:23  iter: 95679  total_loss: 1.017  loss_cls: 0.2267  loss_box_reg: 0.3483  loss_mask: 0.2453  loss_rpn_cls: 0.0707  loss_rpn_loc: 0.1675  time: 0.2237  data_time: 0.0074  lr: 0.00125  max_mem: 16883M
[12/30 03:45:18 d2.utils.events]:  eta: 0:02:19  iter: 95699  total_loss: 1.08  loss_cls: 0.2543  loss_box_reg: 0.3381  loss_mask: 0.2429  loss_rpn_cls: 0.05703  loss_rpn_loc: 0.1529  time: 0.2237  data_time: 0.0245  lr: 0.00125  max_mem: 16883M
[12/30 03:45:22 d2.utils.events]:  eta: 0:02:14  iter: 95719  total_loss: 1.175  loss_cls: 0.3113  loss_box_reg: 0.3856  loss_mask: 0.2728  loss_rpn_cls: 0.08695  loss_rpn_loc: 0.1644  time: 0.2237  data_time: 0.0080  lr: 0.00125  max_mem: 16883M
[12/30 03:45:27 d2.utils.events]:  eta: 0:02:10  iter: 95739  total_loss: 1.221  loss_cls: 0.2891  loss_box_reg: 0.3882  loss_mask: 0.2765  loss_rpn_cls: 0.07242  loss_rpn_loc: 0.16  time: 0.2237  data_time: 0.0072  lr: 0.00125  max_mem: 16883M
[12/30 03:45:32 

[12/30 03:47:43 d2.utils.events]:  eta: 0:00:00  iter: 96339  total_loss: 1.096  loss_cls: 0.2714  loss_box_reg: 0.3186  loss_mask: 0.2467  loss_rpn_cls: 0.07908  loss_rpn_loc: 0.1489  time: 0.2237  data_time: 0.0067  lr: 0.00125  max_mem: 16883M
[12/30 03:47:44 d2.engine.hooks]: Overall training speed: 89114 iterations in 5:32:16 (0.2237 s / it)
[12/30 03:47:44 d2.engine.hooks]: Total training time: 6:16:12 (0:43:55 on hooks)
[12/30 03:47:46 d2.data.datasets.coco]: Loading livecell/livecell_annotations_val.json takes 2.61 seconds.
WARNING [12/30 03:47:46 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/30 03:47:46 d2.data.datasets.coco]: Loaded 570 images in COCO format from livecell/livecell_annotations_val.json
[12/30 03:47:47 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/30 03:47:47 d2.data.comm

## Train on dataset on challenge data

Now that we pretrained the model on livecell we can train the model on the dataset provided by the challenge. We defined the model again for the two different sized backends: ResNet50 and ResNet101. As the pretraining with LiveCell worked best with the ResNet50 we went with it here too.

In addition we defined a parameter `classNum` which when defined only uses one celltype, the one which is associated with the index number, for training. This functionality we used to train three different models to build an **ensemble classifier**. As the Detectron2 model only showed pretty bad results when training on single classes, we did not use the ensemble classifier approach in the end.

In [14]:
def trainModel(classNum=None, withWeights=False):
  torch.cuda.empty_cache()
  cfg = get_cfg()
  cfg.INPUT.MASK_FORMAT='bitmask'

  cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
  cfg.SOLVER.IMS_PER_BATCH = 2
  if classNum is None:
    cfg.DATASETS.TRAIN = ("sartorius_train",)
    cfg.DATASETS.TEST = ("sartorius_val",)
    cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch
  else:
    cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get(f'sartorius_train_{classNum}')) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch
    cfg.DATASETS.TRAIN = (f"sartorius_train_{classNum}",)
    cfg.DATASETS.TEST = (f"sartorius_val_{classNum}",)

  cfg.DATALOADER.NUM_WORKERS = 2
  if not withWeights:
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
  else:
    cfg.MODEL.WEIGHTS = '/livecell/model_final.pth'
  cfg.SOLVER.BASE_LR = 0.01 * 2 / 16
  cfg.SOLVER.MAX_ITER = 10000
  cfg.SOLVER.CHECKPOINT_PERIOD = 300
  cfg.SOLVER.STEPS = []
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
  if (classNum is None):
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
  else:
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS=False
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
  cfg.OUTPUT_DIR = './output'
  cfg.OUTPUT_DIR = os.path.join(cfg.OUTPUT_DIR, f'/{classNum}')
  print(cfg.OUTPUT_DIR)
  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = Trainer(cfg) 
  trainer.resume_or_load(resume=False)
  trainer.train()

In [16]:
def trainModel50():
  torch.cuda.empty_cache()
  cfg = get_cfg()
  cfg.INPUT.MASK_FORMAT='bitmask'

  cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.DATASETS.TRAIN = ("sartorius_train",)
  cfg.DATASETS.TEST = ("sartorius_val",)
  cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

  cfg.DATALOADER.NUM_WORKERS = 2
  # Ise the pretrained weights from LiveCell
  cfg.MODEL.WEIGHTS = './output/livecell50/model_final.pth'
  cfg.SOLVER.BASE_LR = 0.01 * 2 / 16
  cfg.SOLVER.MAX_ITER = 50000
  cfg.SOLVER.CHECKPOINT_PERIOD = 300
  cfg.SOLVER.STEPS = []
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
  cfg.OUTPUT_DIR = './output50'
  print(cfg.OUTPUT_DIR)
  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = Trainer(cfg) 
  trainer.resume_or_load(resume=True)
  trainer.train()

In [17]:
trainModel50()

./output50
[12/30 11:08:48 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )

[12/30 11:08:48 d2.data.build]: Removed 0 images with no usable annotations. 545 images left.
[12/30 11:08:48 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[12/30 11:08:48 d2.data.build]: Using training sampler TrainingSampler
[12/30 11:08:48 d2.data.common]: Serializing 545 elements to byte tensors and concatenating them all ...
[12/30 11:08:48 d2.data.common]: Serialized dataset takes 7.77 MiB
[12/30 11:08:48 d2.engine.hooks]: Loading scheduler from state_dict ...
[12/30 11:08:48 d2.engine.train_loop]: Starting training from iteration 8100


/opt/conda/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/30 11:08:59 d2.utils.events]:  eta: 2:39:19  iter: 8119  total_loss: 1.005  loss_cls: 0.2023  loss_box_reg: 0.2744  loss_mask: 0.2495  loss_rpn_cls: 0.05173  loss_rpn_loc: 0.1976  time: 0.5341  data_time: 0.3167  lr: 0.00125  max_mem: 4229M
[12/30 11:09:13 d2.utils.events]:  eta: 3:42:30  iter: 8139  total_loss: 1.109  loss_cls: 0.1953  loss_box_reg: 0.3039  loss_mask: 0.2641  loss_rpn_cls: 0.07148  loss_rpn_loc: 0.1996  time: 0.6105  data_time: 0.4632  lr: 0.00125  max_mem: 4273M
[12/30 11:09:26 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/30 11:09:26 d2.data.common]: Serializing 61 elements to byte tensors and concatenating them all ...
[12/30 11:09:26 d2.data.common]: Serialized dataset takes 0.71 MiB
[12/30 11:09:26 d2.evaluation.evaluator]: Start inference on 61 batches
[12/30 11:09:26 d2.evaluation.evaluator]: Inference done 11/61. Dataloading: 0.0007 s/it

[12/30 11:14:21 d2.utils.events]:  eta: 3:05:45  iter: 8599  total_loss: 1.09  loss_cls: 0.2111  loss_box_reg: 0.2555  loss_mask: 0.2601  loss_rpn_cls: 0.06863  loss_rpn_loc: 0.2071  time: 0.6491  data_time: 0.5081  lr: 0.00125  max_mem: 6363M
[12/30 11:14:34 d2.utils.events]:  eta: 3:04:24  iter: 8619  total_loss: 0.9754  loss_cls: 0.1716  loss_box_reg: 0.2256  loss_mask: 0.2844  loss_rpn_cls: 0.05925  loss_rpn_loc: 0.194  time: 0.6505  data_time: 0.4672  lr: 0.00125  max_mem: 6363M
[12/30 11:14:46 d2.utils.events]:  eta: 3:03:54  iter: 8639  total_loss: 1.105  loss_cls: 0.1773  loss_box_reg: 0.2622  loss_mask: 0.2615  loss_rpn_cls: 0.07716  loss_rpn_loc: 0.2113  time: 0.6485  data_time: 0.3866  lr: 0.00125  max_mem: 6363M
[12/30 11:14:58 d2.utils.events]:  eta: 3:04:13  iter: 8659  total_loss: 1.105  loss_cls: 0.2295  loss_box_reg: 0.2991  loss_mask: 0.254  loss_rpn_cls: 0.09088  loss_rpn_loc: 0.2075  time: 0.6456  data_time: 0.3455  lr: 0.00125  max_mem: 6363M
[12/30 11:15:13 d2.uti

...

[12/30 17:11:26 d2.utils.events]:  eta: 0:40:49  iter: 41039  total_loss: 0.9011  loss_cls: 0.1524  loss_box_reg: 0.2144  loss_mask: 0.2718  loss_rpn_cls: 0.03843  loss_rpn_loc: 0.1683  time: 0.6449  data_time: 0.4411  lr: 0.00125  max_mem: 6710M
[12/30 17:11:39 d2.utils.events]:  eta: 0:40:34  iter: 41059  total_loss: 0.8067  loss_cls: 0.1327  loss_box_reg: 0.1685  loss_mask: 0.2406  loss_rpn_cls: 0.04017  loss_rpn_loc: 0.1321  time: 0.6449  data_time: 0.4386  lr: 0.00125  max_mem: 6710M
[12/30 17:11:48 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/30 17:11:48 d2.data.common]: Serializing 61 elements to byte tensors and concatenating them all ...
[12/30 17:11:48 d2.data.common]: Serialized dataset takes 0.71 MiB
[12/30 17:11:48 d2.evaluation.evaluator]: Start inference on 61 batches
[12/30 17:11:49 d2.evaluation.evaluator]: Inference done 11/61. Dataloading: 0.0465 

[12/30 17:16:38 d2.utils.events]:  eta: 0:37:41  iter: 41519  total_loss: 0.9652  loss_cls: 0.2162  loss_box_reg: 0.3114  loss_mask: 0.2641  loss_rpn_cls: 0.06759  loss_rpn_loc: 0.1843  time: 0.6447  data_time: 0.5656  lr: 0.00125  max_mem: 6710M
[12/30 17:16:47 d2.utils.events]:  eta: 0:37:23  iter: 41539  total_loss: 0.7278  loss_cls: 0.114  loss_box_reg: 0.1872  loss_mask: 0.228  loss_rpn_cls: 0.02784  loss_rpn_loc: 0.1478  time: 0.6445  data_time: 0.2353  lr: 0.00125  max_mem: 6710M
[12/30 17:17:05 d2.utils.events]:  eta: 0:37:17  iter: 41559  total_loss: 0.9298  loss_cls: 0.172  loss_box_reg: 0.1895  loss_mask: 0.2701  loss_rpn_cls: 0.06371  loss_rpn_loc: 0.2006  time: 0.6447  data_time: 0.7171  lr: 0.00125  max_mem: 6710M
[12/30 17:17:22 d2.utils.events]:  eta: 0:36:35  iter: 41579  total_loss: 0.8998  loss_cls: 0.1417  loss_box_reg: 0.1858  loss_mask: 0.2848  loss_rpn_cls: 0.05656  loss_rpn_loc: 0.1888  time: 0.6448  data_time: 0.5864  lr: 0.00125  max_mem: 6710M
[12/30 17:17:36

[12/30 17:21:50 d2.utils.events]:  eta: 0:36:06  iter: 41999  total_loss: 0.8697  loss_cls: 0.1561  loss_box_reg: 0.2104  loss_mask: 0.2458  loss_rpn_cls: 0.06452  loss_rpn_loc: 0.1825  time: 0.6445  data_time: 0.5796  lr: 0.00125  max_mem: 6710M
[12/30 17:22:07 d2.utils.events]:  eta: 0:36:05  iter: 42019  total_loss: 1.036  loss_cls: 0.1985  loss_box_reg: 0.2985  loss_mask: 0.2553  loss_rpn_cls: 0.03756  loss_rpn_loc: 0.1566  time: 0.6446  data_time: 0.6176  lr: 0.00125  max_mem: 6710M
[12/30 17:22:22 d2.utils.events]:  eta: 0:35:52  iter: 42039  total_loss: 0.8463  loss_cls: 0.1441  loss_box_reg: 0.1863  loss_mask: 0.2391  loss_rpn_cls: 0.0455  loss_rpn_loc: 0.1817  time: 0.6446  data_time: 0.5123  lr: 0.00125  max_mem: 6710M
[12/30 17:22:38 d2.utils.events]:  eta: 0:35:54  iter: 42059  total_loss: 0.9211  loss_cls: 0.1495  loss_box_reg: 0.2  loss_mask: 0.2711  loss_rpn_cls: 0.054  loss_rpn_loc: 0.1781  time: 0.6447  data_time: 0.5673  lr: 0.00125  max_mem: 6710M
[12/30 17:22:49 d2.

[12/30 17:27:23 d2.utils.events]:  eta: 0:33:15  iter: 42479  total_loss: 0.9058  loss_cls: 0.157  loss_box_reg: 0.1967  loss_mask: 0.2754  loss_rpn_cls: 0.06072  loss_rpn_loc: 0.1987  time: 0.6449  data_time: 0.5792  lr: 0.00125  max_mem: 6710M
[12/30 17:27:35 d2.utils.events]:  eta: 0:33:11  iter: 42499  total_loss: 0.7518  loss_cls: 0.1155  loss_box_reg: 0.1841  loss_mask: 0.2406  loss_rpn_cls: 0.0304  loss_rpn_loc: 0.1551  time: 0.6449  data_time: 0.3950  lr: 0.00125  max_mem: 6710M
[12/30 17:27:42 d2.engine.hooks]: Overall training speed: 34409 iterations in 6:09:49 (0.6449 s / it)
[12/30 17:27:42 d2.engine.hooks]: Total training time: 6:18:52 (0:09:02 on hooks)
[12/30 17:27:42 d2.utils.events]:  eta: 0:33:00  iter: 42511  total_loss: 0.4376  loss_cls: 0.05312  loss_box_reg: 0.07458  loss_mask: 0.1203  loss_rpn_cls: 0.02754  loss_rpn_loc: 0.1551  time: 0.6449  data_time: 0.3632  lr: 0.00125  max_mem: 6710M


KeyboardInterrupt: 

In [ ]:
trainModelLiveCell()

[12/27 12:07:41 d2.data.datasets.coco]: Loading /content/livecell/livecell_annotations_train.json takes 21.98 seconds.
WARNING [12/27 12:07:41 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/27 12:07:42 d2.data.datasets.coco]: Loaded 3253 images in COCO format from /content/livecell/livecell_annotations_train.json
[12/27 12:08:01 d2.data.datasets.coco]: Loading /content/livecell/livecell_annotations_test.json takes 8.76 seconds.
WARNING [12/27 12:08:01 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/27 12:08:01 d2.data.datasets.coco]: Loaded 1564 images in COCO format from /content/livecell/livecell_annotations_test.json
/livecell
[12/27 12:08:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1)

model_final_a3ec72.pkl: 254MB [00:12, 20.6MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (9, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (9,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (32, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (32,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to

[12/27 12:09:37 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/27 12:09:48 d2.utils.events]:  eta: 1:13:29  iter: 19  total_loss: 5.291  loss_cls: 1.951  loss_box_reg: 0.421  loss_mask: 0.6924  loss_rpn_cls: 1.823  loss_rpn_loc: 0.3426  time: 0.4565  data_time: 0.0536  lr: 9.9905e-05  max_mem: 9000M
[12/27 12:09:57 d2.utils.events]:  eta: 1:15:26  iter: 39  total_loss: 2.924  loss_cls: 1.153  loss_box_reg: 0.4876  loss_mask: 0.6856  loss_rpn_cls: 0.2815  loss_rpn_loc: 0.2853  time: 0.4726  data_time: 0.0240  lr: 0.0001998  max_mem: 11392M
[12/27 12:10:07 d2.utils.events]:  eta: 1:14:51  iter: 59  total_loss: 2.906  loss_cls: 1.066  loss_box_reg: 0.5713  loss_mask: 0.6728  loss_rpn_cls: 0.2219  loss_rpn_loc: 0.2409  time: 0.4668  data_time: 0.0135  lr: 0.0002997  max_mem: 11392M
[12/27 12:10:16 d2.utils.events]:  eta: 1:14:24  iter: 79  total_loss: 2.693  loss_cls: 1.018  loss_box_reg: 0.5283  loss_mask: 0.6462  loss_rpn_cls: 0.2191  loss_rpn_loc: 0.2406  time: 0.4622  data_time: 0.0137  lr: 0.00039961  max_mem: 11392M
[12/27 12:10:25 d2.utils.

In [ ]:
torch.cuda.empty_cache()
trainModel(0, True)

/0
[12/27 14:05:07 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in the 

[12/27 14:05:08 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This ty

[12/27 14:05:16 d2.utils.events]:  eta: 1:06:16  iter: 19  total_loss: 1.286  loss_cls: 0.6369  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.4405  loss_rpn_loc: 0.1593  time: 0.3868  data_time: 0.0587  lr: 2.4976e-05  max_mem: 13868M
[12/27 14:05:24 d2.utils.events]:  eta: 1:06:03  iter: 39  total_loss: 1.081  loss_cls: 0.5504  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.3244  loss_rpn_loc: 0.1659  time: 0.3879  data_time: 0.0123  lr: 4.9951e-05  max_mem: 13868M
[12/27 14:05:32 d2.utils.events]:  eta: 1:05:47  iter: 59  total_loss: 0.8182  loss_cls: 0.4056  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.2942  loss_rpn_loc: 0.1271  time: 0.3904  data_time: 0.0150  lr: 7.4926e-05  max_mem: 13868M
[12/27 14:05:40 d2.utils.events]:  eta: 1:05:39  iter: 79  total_loss: 0.6276  loss_cls: 0.281  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.2424  loss_rpn_loc: 0.1368  time: 0.3927  data_time: 0.0184  lr: 9.9901e-05  max_mem: 13868M
[12/27 14:05:48 d2.utils.events]:  eta: 1:05:31  iter: 

In [ ]:
torch.cuda.empty_cache()
trainModel(1, True)


/1
[12/27 15:13:40 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in the 

[12/27 15:13:40 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This ty

[12/27 15:13:49 d2.utils.events]:  eta: 1:04:38  iter: 19  total_loss: 1.355  loss_cls: 0.6788  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.3977  loss_rpn_loc: 0.2297  time: 0.3964  data_time: 0.0646  lr: 2.4976e-05  max_mem: 13868M
[12/27 15:13:57 d2.utils.events]:  eta: 1:04:30  iter: 39  total_loss: 1.133  loss_cls: 0.5656  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.3983  loss_rpn_loc: 0.1321  time: 0.3892  data_time: 0.0112  lr: 4.9951e-05  max_mem: 13868M
[12/27 15:14:05 d2.utils.events]:  eta: 1:04:23  iter: 59  total_loss: 0.9338  loss_cls: 0.4233  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.3861  loss_rpn_loc: 0.1207  time: 0.3878  data_time: 0.0136  lr: 7.4926e-05  max_mem: 13868M
[12/27 15:14:13 d2.utils.events]:  eta: 1:04:14  iter: 79  total_loss: 0.7224  loss_cls: 0.253  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.274  loss_rpn_loc: 0.1496  time: 0.3887  data_time: 0.0097  lr: 9.9901e-05  max_mem: 13868M
[12/27 15:14:20 d2.utils.events]:  eta: 1:04:04  iter: 9

In [ ]:
torch.cuda.empty_cache()
trainModel(2, True)

/2
[12/27 16:21:53 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in the 

[12/27 16:21:54 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This ty

[12/27 16:22:02 d2.utils.events]:  eta: 1:04:43  iter: 19  total_loss: 1.319  loss_cls: 0.6673  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.3655  loss_rpn_loc: 0.1734  time: 0.3827  data_time: 0.0465  lr: 2.4976e-05  max_mem: 13868M
[12/27 16:22:10 d2.utils.events]:  eta: 1:03:59  iter: 39  total_loss: 1.15  loss_cls: 0.5846  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.3569  loss_rpn_loc: 0.149  time: 0.3818  data_time: 0.0123  lr: 4.9951e-05  max_mem: 13868M
[12/27 16:22:18 d2.utils.events]:  eta: 1:04:05  iter: 59  total_loss: 0.9317  loss_cls: 0.4033  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.363  loss_rpn_loc: 0.1499  time: 0.3840  data_time: 0.0098  lr: 7.4926e-05  max_mem: 13868M
[12/27 16:22:25 d2.utils.events]:  eta: 1:03:50  iter: 79  total_loss: 0.6813  loss_cls: 0.2562  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.2673  loss_rpn_loc: 0.1413  time: 0.3804  data_time: 0.0110  lr: 9.9901e-05  max_mem: 13868M
[12/27 16:22:33 d2.utils.events]:  eta: 1:03:40  iter: 99

In [ ]:
torch.cuda.empty_cache()
trainModel(None, True)

/None
[12/27 17:52:03 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
    

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in th

[12/27 17:52:04 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This ty

[12/27 17:52:29 d2.utils.events]:  eta: 1:22:22  iter: 19  total_loss: 2.822  loss_cls: 1.252  loss_box_reg: 0.2487  loss_mask: 0.693  loss_rpn_cls: 0.314  loss_rpn_loc: 0.2942  time: 1.2386  data_time: 0.8289  lr: 2.4976e-05  max_mem: 13868M
[12/27 17:52:59 d2.utils.events]:  eta: 1:26:42  iter: 39  total_loss: 2.741  loss_cls: 1.163  loss_box_reg: 0.3582  loss_mask: 0.6835  loss_rpn_cls: 0.308  loss_rpn_loc: 0.2912  time: 1.3655  data_time: 1.0235  lr: 4.9951e-05  max_mem: 13868M
[12/27 17:53:19 d2.utils.events]:  eta: 1:23:26  iter: 59  total_loss: 2.342  loss_cls: 0.9056  loss_box_reg: 0.09632  loss_mask: 0.639  loss_rpn_cls: 0.3446  loss_rpn_loc: 0.3494  time: 1.2310  data_time: 0.5543  lr: 7.4926e-05  max_mem: 13868M
[12/27 17:53:39 d2.utils.events]:  eta: 1:21:50  iter: 79  total_loss: 2.023  loss_cls: 0.6164  loss_box_reg: 0.2543  loss_mask: 0.6033  loss_rpn_cls: 0.2547  loss_rpn_loc: 0.29  time: 1.1718  data_time: 0.5555  lr: 9.9901e-05  max_mem: 13868M
[12/27 17:54:09 d2.util

In [ ]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

### Inference

The following code loads the trained models, which were trained on only one class, and puts them in a list.

In [ ]:
models = []
MIN_PIXELS = [75, 150, 75]


for classNum in range(3):
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
  cfg.INPUT.MASK_FORMAT='bitmask'
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
  cfg.MODEL.WEIGHTS = f'/{classNum}/model_final.pth'
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.01
  cfg.TEST.DETECTIONS_PER_IMAGE = 1000
  models.append(DefaultPredictor(cfg))



Additionally we also load the trained model which was used after training with LiveCell.

In [19]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.WEIGHTS = f'./output50/model_0042299.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.TEST.DETECTIONS_PER_IMAGE = 1000
allModel = DefaultPredictor(cfg)

The following code uses the ensemble classifier models to create predictions by concatinating all. When running this all models did not predict any instances. Thus we were not able to successfully build an ensemble classifier using three detectron instances.

In [20]:
# Parts of this code is taken from
# https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-inference
from detectron2.structures import Instances
def predictMask(name):
  im = cv2.imread(str(name))
  instances = None
  # Remove comment to debug predictions
  # for model in models:
  #   print(model(im))
  # return
  for classNum, model in enumerate(models):
    pred = model(im)
    pred['instances'].pred_classes *= classNum + 1
    if instances is None:
      instances = pred['instances']
    else:
      instances = Instances.cat(instances, pred['instances'])
    take = pred['instances'].scores >= THRESHOLDS[classNum]
    pred_masks = pred['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    res = []
    used = np.zeros(im.shape[:2], dtype=int) 
    for mask in pred_masks:
        mask = mask * (1-used)
        if mask.sum() >= MIN_PIXELS[classNum]: # skip predictions with small area
            used += mask
            res.append(rle_encode(mask))
    return res

This function builds predictions using the single model for all cell types.

In [21]:
def predictMaskSingle(name):
  im = cv2.imread(str(name))
  pred = allModel(im)
  take = pred['instances'].scores >= THRESHOLDS[classNum]
  pred_masks = pred['instances'].pred_masks[take]
  pred_masks = pred_masks.cpu().numpy()
  res = []
  used = np.zeros(im.shape[:2], dtype=int) 
  for mask in pred_masks:
      mask = mask * (1-used)
      if mask.sum() >= MIN_PIXELS[classNum]: # skip predictions with small area
          used += mask
          res.append(rle_encode(mask))
  return res

### Scoring functions

The next functions compute an average MaP-IoU score for all the data in the Sartorius dataset.

In [22]:
# Define default thresholds. Only instances where the mdodels logits are higher than the threshold
# are kept.
# Part of this code is taken and adapted from 
# https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-training
THRESHOLDS = [.15, .15, .15]
def scoreWithThreshold(pred, targ):
  enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
  enc_targs = list(map(lambda x:x['segmentation'], targ))
  ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
  prec = []
  for t in np.arange(0.5, 1.0, 0.05):
      tp, fp, fn = precision_at(t, ious)
      p = tp / (tp + fp + fn) if (tp + fp + fn) != 0 else 0
      prec.append(p)
  return np.mean(prec)

def evaluateSingleModel():
  dataset_dicts = DatasetCatalog.get('sartorius_val')
  ious_res = []
  for X in tqdm(dataset_dicts):
    img = cv2.imread(X["file_name"])
    pred = allModel(img)
    if len(pred['instances']) == 0:
      ious_res.append(0)
      continue
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    take = pred['instances'].scores >= THRESHOLDS[pred_class]
    pred_masks = pred['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    res = []
    used = np.zeros(img.shape[:2], dtype=int) 
    for mask in pred_masks:
        mask = mask * (1-used)
        if mask.sum() >= MIN_PIXELS[pred_class]: # skip predictions with small area
            used += mask
            res.append(mask.astype(np.uint8))
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in res]
    enc_targs = list(map(lambda x:x['segmentation'], X["annotations"]))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn) if (tp + fp + fn) != 0 else 0
        prec.append(p)
    ious_res.append(np.mean(prec))
  return np.mean(ious)

In [26]:
cat_ids

{'shsy5y': 1, 'astro': 2, 'cort': 3}

### Find best Thresholds

We perform a greedy search for the best thresholds by increasing each and scoring the model with the increased threshold. We use the computed minimal pixels for each cell type.

In [ ]:
MIN_PIXELS = [13,18,26] # shsy5y, astro, cort
for i in list(range(3)):
    thresh = 0.01
    bestThresh = 0.01
    currentMax = -1
    while thresh <= 0.8:
        THRESHOLDS[i] = thresh
        scored = evaluateSingleModel()
        if scored > currentMax:
            bestThresh = thresh
            currentMax = scored
        thresh += 0.02
    THRESHOLDS[i] = bestThresh
THRESHOLDS

100% 61/61 [00:05<00:00, 11.20it/s]
100% 61/61 [00:05<00:00, 11.35it/s]
100% 61/61 [00:05<00:00, 11.30it/s]
100% 61/61 [00:05<00:00, 11.27it/s]
100% 61/61 [00:05<00:00, 11.32it/s]
100% 61/61 [00:05<00:00, 11.27it/s]
100% 61/61 [00:05<00:00, 11.28it/s]
100% 61/61 [00:05<00:00, 11.23it/s]
100% 61/61 [00:05<00:00, 11.27it/s]
100% 61/61 [00:05<00:00, 11.29it/s]
100% 61/61 [00:05<00:00, 11.29it/s]
100% 61/61 [00:05<00:00, 11.31it/s]
100% 61/61 [00:05<00:00, 11.34it/s]
100% 61/61 [00:05<00:00, 11.38it/s]
 56% 34/61 [00:02<00:02, 12.14it/s]

In [ ]:
TRESHOLDS

Check if RLE predictions are made.

In [ ]:

predictMaskSingle('train/0140b3c8f445.png')

['215857 3 216539 4 216556 12 217241 34 217944 37 218648 39 219353 39 220059 39 220766 37 221480 29 222189 25 222896 23 223602 23 224307 22 225013 21 225718 21 226424 20 227130 19 227835 19 228542 17 229248 16 229955 15 230663 13 231369 12 232075 11 232781 11 233486 11 234192 11 234898 10 235604 9 236309 10 237014 10 237720 10 238425 10 239131 10 239836 10 240542 10 241248 9 241953 9 242658 10 243363 10 244069 9 244775 9 245481 8 246185 9 246890 10 247595 10 248302 8 249007 8 249712 9 250417 9 251122 9 251827 10 252533 9 253238 9 253943 10 254649 9 255354 10 256059 10 256764 10 257469 10 258174 10 258880 8 259586 6 260291 2',
 '298493 1 299197 1 299900 2 300603 3 301306 4 302010 4 302713 5 303416 6 304120 6 304823 7 305526 8 306230 8 306933 9 307636 10 308339 10 309042 11 309745 11 310449 11 311152 11 311856 11 312559 11 313263 11 313967 11 314670 12 315374 12 316078 12 316783 11 317487 11 318191 11 318895 11 319599 11 320304 10 321008 10 321713 9 322417 9 323121 9 323826 7 324530 7 32

Now we can upload the weights and config to kaggle to make the predictions.